# 彩色图片分类

## 加载相关库

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, optimizers
import matplotlib.pyplot as plt

Init Plugin
Init Graph Optimizer
Init Kernel


## 导入数据

In [2]:
(train_x, train_y), (test_x, test_y) = datasets.cifar10.load_data()

## 归一化

In [3]:
train_x, test_x = train_x / 255.0, test_x / 255.0

print("train_data: ", train_x.shape, train_y.shape)
print("test_data: ", test_x.shape, test_y.shape)

train_data:  (50000, 32, 32, 3) (50000, 1)
test_data:  (10000, 32, 32, 3) (10000, 1)


## 建模

In [4]:
class CNN(tf.keras.Model):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_1 = layers.Conv2D(32, (3, 3), activation='relu')
        self.conv_2 = layers.Conv2D(64, (3, 3), activation='relu')
        self.conv_3 = layers.Conv2D(64, (3, 3), activation='relu')

        self.max_pooling = layers.MaxPooling2D((2, 2))
        self.flatten = layers.Flatten()
        
        self.linear_1 = layers.Dense(64, activation='relu')
        self.linear_2 = layers.Dense(10)

    def call(self, x):
        x = self.conv_1(x)
        x = self.max_pooling(x)
        x = self.conv_2(x)
        x = self.max_pooling(x)
        x = self.conv_3(x)
        x = self.flatten(x)
        x = self.linear_1(x)
        x = self.linear_2(x)
        return x

model = CNN()

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2021-08-26 23:05:53.923106: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-08-26 23:05:53.923550: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
train_x = tf.convert_to_tensor(train_x, dtype=tf.float32)
test_x = tf.convert_to_tensor(test_x, dtype=tf.float32)

train_y = tf.convert_to_tensor(train_y)
test_y = tf.convert_to_tensor(test_y)


train_db = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(100)
test_db = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(100)

In [6]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = optimizers.Adam()

In [7]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [8]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        prediction = model(images)
        loss = loss_function(labels, prediction)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, prediction)


@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_function(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [9]:
EPOCHS = 10


for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_db:
        train_step(images, labels)

    for test_images, test_labels in test_db:
        test_step(test_images, test_labels)

    template = 'Epoch {}, Loss: {}, Accuracy: {}%, Test Loss: {}, Test Accuracy: {}%'
        
    print(template.format(epoch + 1,
                          train_loss.result(), 
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))

2021-08-26 23:05:55.178815: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-08-26 23:05:55.181161: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-08-26 23:05:55.181641: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-08-26 23:06:01.728387: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1, Loss: 1.6293104887008667, Accuracy: 40.39800262451172%, Test Loss: 1.38679838180542, Test Accuracy: 50.51000213623047%
Epoch 2, Loss: 1.27566659450531, Accuracy: 54.28800582885742%, Test Loss: 1.2001688480377197, Test Accuracy: 56.95000457763672%
Epoch 3, Loss: 1.1287208795547485, Accuracy: 60.2180061340332%, Test Loss: 1.1472587585449219, Test Accuracy: 59.910003662109375%
Epoch 4, Loss: 1.0380208492279053, Accuracy: 63.716007232666016%, Test Loss: 1.0259485244750977, Test Accuracy: 64.29000854492188%
Epoch 5, Loss: 0.9613401889801025, Accuracy: 66.56000518798828%, Test Loss: 0.9838595390319824, Test Accuracy: 65.93000030517578%
Epoch 6, Loss: 0.8983980417251587, Accuracy: 68.73200225830078%, Test Loss: 0.949745237827301, Test Accuracy: 67.2300033569336%
Epoch 7, Loss: 0.8454487919807434, Accuracy: 70.5780029296875%, Test Loss: 0.9266660809516907, Test Accuracy: 68.16000366210938%
Epoch 8, Loss: 0.8037283420562744, Accuracy: 72.16600036621094%, Test Loss: 0.8999237418174744, 